# IT Academy - Data Science with Python
## Sprint 8: Hypothesis Testing
### [Github Hypothesis Testing](https://github.com/jesussantana/Hypothesis-testing)

[![forthebadge made-with-python](http://ForTheBadge.com/images/badges/made-with-python.svg)](https://www.python.org/)  
[![Made withJupyter](https://img.shields.io/badge/Made%20with-Jupyter-orange?style=for-the-badge&logo=Jupyter)](https://jupyter.org/try)  
[![wakatime](https://wakatime.com/badge/github/jesussantana/Hypothesis-testing.svg)](https://wakatime.com/badge/github/jesussantana/Hypothesis-testing)

### Exercise 1: 
  - Grab a sports theme dataset you like and select an attribute from the dataset. Calculate the p-value and say if you reject the null hypothesis by taking a 5% alpha.

In [13]:
import pandas as pd 
import numpy as np
import scipy as sp
import datetime
import warnings
import time
import math

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import IPython as ip

import scipy.stats as stats

mpl.style.use('ggplot')
mpl.rc('font', family='Noto Sans CJK TC')
ip.display.set_matplotlib_formats('svg')

warnings.filterwarnings('ignore')
sns.set_theme(style='darkgrid', palette='deep')

In [14]:
np.random.seed(20180701+3)

In [15]:
pd.set_option('display.max_columns', None)

path = '../data/'
file = 'raw/MLB_Stats.csv'

df_raw = pd.read_csv(path+file)

In [16]:
df = df_raw.copy()

In [17]:
df.head()

,playerID,Player Name,weight,height,bats,throws,Season,League,teamID,Team,Franchise,G,AB,R,H,2B,3B,HR,RBI,SB,BB,SO,salary,IBB,HBP,SH,SF,PA
0,aardsda01,David Aardsma,215,75,R,R,2004,NL,SFN,San Francisco Giants,San Francisco Giants,11,0,0,0,0,0,0,0,0,0,0,300000,0,0,0,0,0
1,aardsda01,David Aardsma,215,75,R,R,2007,AL,CHA,Chicago White Sox,Chicago White Sox,25,0,0,0,0,0,0,0,0,0,0,387500,0,0,0,0,0
2,aardsda01,David Aardsma,215,75,R,R,2008,AL,BOS,Boston Red Sox,Boston Red Sox,47,1,0,0,0,0,0,0,0,0,1,403250,0,0,0,0,1
3,aardsda01,David Aardsma,215,75,R,R,2009,AL,SEA,Seattle Mariners,Seattle Mariners,73,0,0,0,0,0,0,0,0,0,0,419000,0,0,0,0,0
4,aardsda01,David Aardsma,215,75,R,R,2012,AL,NYA,New York Yankees,New York Yankees,1,0,0,0,0,0,0,0,0,0,0,500000,0,0,0,0,0


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28293 entries, 0 to 28292
Data columns (total 28 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   playerID     28293 non-null  object
 1   Player Name  28293 non-null  object
 2   weight       28293 non-null  int64 
 3   height       28293 non-null  int64 
 4   bats         28293 non-null  object
 5   throws       28293 non-null  object
 6   Season       28293 non-null  int64 
 7   League       28293 non-null  object
 8   teamID       28293 non-null  object
 9   Team         28293 non-null  object
 10  Franchise    28293 non-null  object
 11  G            28293 non-null  int64 
 12  AB           28293 non-null  int64 
 13  R            28293 non-null  int64 
 14  H            28293 non-null  int64 
 15  2B           28293 non-null  int64 
 16  3B           28293 non-null  int64 
 17  HR           28293 non-null  int64 
 18  RBI          28293 non-null  int64 
 19  SB           28293 non-nu

In [19]:
df.shape

(28293, 28)

In [20]:
df.describe().round(3)

,weight,height,Season,G,AB,R,H,2B,3B,HR,RBI,SB,BB,SO,salary,IBB,HBP,SH,SF,PA
count,28293.000,28293.000,28293.000,28293.000,28293.000,28293.000,28293.000,28293.000,28293.000,28293.000,28293.000,28293.000,28293.000,28293.000,2.829300e+04,28293.000,28293.000,28293.000,28293.000,28293.000
mean,199.049,73.500,2000.938,63.934,161.880,21.787,42.625,8.221,0.909,4.777,20.784,3.108,15.802,30.376,2.112965e+06,1.326,1.451,1.535,1.367,183.360
std,22.654,2.287,8.901,47.285,199.407,30.134,56.201,11.401,1.832,8.384,29.705,7.495,23.036,37.624,3.440239e+06,3.161,2.782,2.748,2.208,226.217
min,140.000,66.000,1985.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.090000e+04,0.000,0.000,0.000,0.000,0.000
25%,185.000,72.000,1994.000,26.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000000e+05,0.000,0.000,0.000,0.000,1.000
50%,195.000,74.000,2001.000,50.000,57.000,4.000,10.000,2.000,0.000,0.000,4.000,0.000,3.000,15.000,6.000000e+05,0.000,0.000,0.000,0.000,66.000
75%,215.000,75.000,2009.000,101.000,298.000,37.000,76.000,14.000,1.000,6.000,34.000,2.000,25.000,50.000,2.425000e+06,1.000,2.000,2.000,2.000,337.000
max,315.000,83.000,2016.000,163.000,716.000,152.000,262.000,59.000,23.000,73.000,165.000,110.000,232.000,223.000,3.300000e+07,120.000,35.000,39.000,17.000,783.000


### Normality Tests

- Shapiro-Wilk Test

In [21]:
stat, p = stats.shapiro(df.R)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Probably Gaussian')
else:
	print('Probably not Gaussian')

stat=0.755, p=0.000
Probably not Gaussian


- D’Agostino’s K^2 Test

In [22]:
stat, p = stats.normaltest(df.R)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Probably Gaussian')
else:
	print('Probably not Gaussian')

stat=5783.676, p=0.000
Probably not Gaussian


- Anderson-Darling Test

In [23]:
result = stats.anderson(df.R)
print('stat=%.3f' % (result.statistic))
for i in range(len(result.critical_values)):
	sl, cv = result.significance_level[i], result.critical_values[i]
	if result.statistic < cv:
		print('Probably Gaussian at the %.1f%% level' % (sl))
	else:
		print('Probably not Gaussian at the %.1f%% level' % (sl))

stat=2759.009
Probably not Gaussian at the 15.0% level
Probably not Gaussian at the 10.0% level
Probably not Gaussian at the 5.0% level
Probably not Gaussian at the 2.5% level
Probably not Gaussian at the 1.0% level


### Correlation Tests

- Pearson’s Correlation Coefficient

In [24]:
stat, p = stats.pearsonr(df.R, df.H)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Probably independent')
else:
	print('Probably dependent')

stat=0.974, p=0.000
Probably dependent


- Spearman’s Rank Correlation

In [25]:
stat, p = stats.spearmanr(df.R, df.H)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Probably independent')
else:
	print('Probably dependent')

stat=0.978, p=0.000
Probably dependent


- Kendall’s Rank Correlation

In [26]:
stat, p = stats.kendalltau(df.R, df.H)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Probably independent')
else:
	print('Probably dependent')

stat=0.914, p=0.000
Probably dependent


- Chi-Squared Test

In [ ]:
stat, p, dof, expected = stats.chi2_contingency(df.R, df.H)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Probably independent')
else:
	print('Probably dependent')